In [ ]:
from pydub import AudioSegment
from pydub.playback import play
from pydub import AudioSegment
import numpy as np
import os
from tqdm import tqdm
import json
from surfboard.sound import Waveform
from surfboard.feature_extraction import extract_features
from surfboard.feature_extraction import extract_features_from_waveform
import pandas as pd  
from pydub.utils import make_chunks

# extract prosodic features without jitter, shimmer, f0

In [ ]:
path = '/Users/andreibirladeanu/Documents/Data/accoustic_extraction/meal/child_speech/'
os.chdir('/Users/andreibirladeanu/Documents/Data/accoustic_extraction/meal/child_speech/')  #if string starts with slash it is considered absolute
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])

In [ ]:
waveforms = []
chunk_length_ms = 1000
for file in file_names:|
    audio = AudioSegment.from_file(os.path.join(path,file)) 
    waveforms.append(make_chunks(audio, chunk_length_ms))

In [ ]:
components = ['loudness', 'loudness_slidingwindow', 'shannon_entropy', 'log_energy', 
             'intensity', 'rms', 'log_energy', 'zerocrossing']
statistics = ['mean', 'std', 'percentile_1_99_range']

In [ ]:
import warnings
warnings.filterwarnings("ignore")
features = []
for participant in tqdm(waveforms):
    temp_list=[]
    for waveform in participant:
        waveform.export("temp.wav", format="wav")
        sound = Waveform(path='temp.wav')
        temp_list.append(extract_features_from_waveform(components, statistics, sound))
        os.remove("temp.wav")
    features.append(temp_list)
print('success')

In [ ]:
indexes = [int(file.split('_')[0]) for file in file_names]

In [ ]:
dfs = []
for participant, index in zip(features, indexes):
    dfs.append(pd.DataFrame(participant))

In [ ]:
for ind in indexes:
    dfs[indexes.index(ind)]['s_id'] = ind

In [ ]:
for df in range(len(dfs)):
    dfs[df] = dfs[df].set_index('s_id')

dfs[1]

In [ ]:
final = pd.concat([df for df in dfs])
final

In [ ]:
os.chdir('/Users/andreibirladeanu/Documents/Data/classification/accoustics/extraction_3.0') 
final.to_excel('meal_cg_prosody.xlsx')

# extract mfcc's 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
features = []
for participant in tqdm(waveforms):
    temp_list=[]
    for waveform in participant:
        waveform.export("temp.wav", format="wav")
        sound = Waveform(path='temp.wav')
        temp_list.append(extract_features([sound], ['mfcc'], ['mean', 'std']))
        os.remove("temp.wav")
    features.append(temp_list)
print('success')

In [ ]:
indexes = [int(file.split('_')[0]) for file in file_names]

In [ ]:
dfs = []
for participant, index in zip(features, indexes):
    temp_list = []
    for feature in participant:
        temp_list.append(np.squeeze(feature.values.T))
    dfs.append(pd.DataFrame(temp_list, columns=features[0][0].columns))

In [ ]:
for ind in indexes:
    dfs[indexes.index(ind)]['s_id'] = ind

In [ ]:
final = pd.concat([df for df in dfs])
final.set_index('s_id', inplace=True)

In [ ]:
final.index, data.index

In [ ]:
os.chdir('/Users/andreibirladeanu/Documents/Data/classification/speech')
data = pd.read_excel('classification.xlsx', engine='openpyxl')

In [ ]:
data = data.set_index('Subject_ID')

In [ ]:
data.index.rename('s_id', inplace=True)

In [ ]:
final_df = final.join(data['label'])

In [ ]:
final_df = final.join(data['label']).drop_duplicates()

In [ ]:
os.chdir('/Users/andreibirladeanu/Documents/Data/classification/accoustics/extraction_3.0')
final_df.to_excel('meal_cg_mfcc.xlsx')

In [ ]:
final

# use parselmouth to extract jitter, shimmer and f1

In [ ]:
import glob
import numpy as np
import pandas as pd
import parselmouth

from parselmouth.praat import call

In [ ]:

# This is the function to measure voice pitch
def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    

    return meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer

In [ ]:
path = '/Users/andreibirladeanu/Documents/Data/accoustic_extraction/meal/child_speech'
os.chdir('/Users/andreibirladeanu/Documents/Data/accoustic_extraction/meal/child_speech')  #if string starts with slash it is considered absolute
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])

In [ ]:
chunk_length_ms = 1000

dfs = []
for file in tqdm(file_names):
    audio = AudioSegment.from_file(os.path.join(path, file))
    file_list = []
    mean_F0_list = []
    sd_F0_list = []
    hnr_list = []
    localJitter_list = []
    localabsoluteJitter_list = []
    rapJitter_list = []
    ppq5Jitter_list = []
    ddpJitter_list = []
    localShimmer_list = []
    localdbShimmer_list = []
    apq3Shimmer_list = []
    aqpq5Shimmer_list = []
    apq11Shimmer_list = []
    ddaShimmer_list = []
    for segment in make_chunks(audio, chunk_length_ms):
        segment.export('temp.wav', format='wav')
        sound = parselmouth.Sound('temp.wav')
        (meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer) = measurePitch(sound, 75, 500, "Hertz")
        file_list.append(file) # make an ID list
        mean_F0_list.append(meanF0) # make a mean F0 list
        sd_F0_list.append(stdevF0) # make a sd F0 list
        hnr_list.append(hnr)
        localJitter_list.append(localJitter)
        localabsoluteJitter_list.append(localabsoluteJitter)
        rapJitter_list.append(rapJitter)
        ppq5Jitter_list.append(ppq5Jitter)
        ddpJitter_list.append(ddpJitter)
        localShimmer_list.append(localShimmer)
        localdbShimmer_list.append(localdbShimmer)
        apq3Shimmer_list.append(apq3Shimmer)
        aqpq5Shimmer_list.append(aqpq5Shimmer)
        apq11Shimmer_list.append(apq11Shimmer)
        ddaShimmer_list.append(ddaShimmer)
    dfs.append(np.column_stack([file_list, mean_F0_list, sd_F0_list, hnr_list, localJitter_list, localabsoluteJitter_list, rapJitter_list, ppq5Jitter_list, ddpJitter_list, localShimmer_list, localdbShimmer_list,
                                apq3Shimmer_list, aqpq5Shimmer_list, apq11Shimmer_list, ddaShimmer_list])) 
print('success')

In [ ]:
waveforms = []
chunk_length_ms = 1000
audio = AudioSegment.from_file('/Users/andreibirladeanu/Documents/Data/accoustic_extraction/play/cg_speech/1043_play_cg.wav') 
print(make_chunks(audio, chunk_length_ms))

In [ ]:
columns=['meanF0Hz', 'stdevF0Hz', 'HNR', 'localJitter', 'localabsoluteJitter', 'rapJitter', 
                                        'ppq5Jitter', 'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'apq5Shimmer', 
                                        'apq11Shimmer', 'ddaShimmer']

In [ ]:
dfs[0][:,0]

In [ ]:
dataframes = []
for df in dfs:
    dataframes.append(pd.DataFrame(df[:,1:], columns = columns, index=df[:,0]))

In [ ]:
dataframes[0]

In [ ]:
final = result = pd.concat([df for df in dataframes])

In [ ]:
os.chdir('/Users/andreibirladeanu/Documents/Data/classification/accoustics/extraction_5.0')
final.to_excel('meal_child_prosody.xlsx')

In [ ]:
final

In [ ]:

df = pd.read_excel('/Users/andreibirladeanu/Documents/Data/classification/accoustics/extraction_3.0/meal_child_prosody.xlsx', index_col=0)

In [ ]:
df = df.drop_duplicates()
df

In [ ]:
os.chdir('/Users/andreibirladeanu/Documents/Data/classification/accoustics/extraction_3.0')

df.to_excel('meal_child_prosody.xlsx')

In [ ]:
df